# Husky glitch exploration

Husky's FPGA includes a small logic analyzer which allows the glitch generation to be visualized.

This is also a companion to test_husky.py, for when visual inspection of glitches is needed.

In [ ]:
SCOPE="OPENADC"
PLATFORM="CWHUSKY"

In [ ]:
import chipwhisperer as cw
scope = cw.scope(name='Husky')

In [ ]:
%run "../Setup_Scripts/Setup_Generic.ipynb"

In [ ]:
scope.clock.clkgen_src = 'system'
scope.clock.clkgen_freq = 10e6
scope.clock.adc_mul = 1

scope.adc.basic_mode = "rising_edge"

scope.trigger.triggers = "tio4"
scope.io.hs2 = "clkgen"

### set up glitch:

In [ ]:
scope.glitch.enabled = True
scope.glitch.clk_src = 'pll'
scope.clock.pll.update_fpga_vco(600e6)
scope.glitch.repeat = 4
scope.glitch.output = 'glitch_only'
scope.glitch.trigger_src = 'manual'
scope.glitch.repeat = 1

In [ ]:
assert scope.glitch.mmcm_locked

### set up LA:

In [ ]:
scope.LA.enabled = True
scope.LA.oversampling_factor = 50
scope.LA.capture_group = 0
scope.LA.trigger_source = "glitch_source"

In [ ]:
assert scope.LA.locked

### Single capture:

In [ ]:
# adjust as you wish:
scope.glitch.offset = 1000
scope.glitch.width = 1000

In [ ]:
scope.glitch.manual_trigger()
glitchout = scope.LA.read_capture(0)
source    = scope.LA.read_capture(1)
mmcm1out  = scope.LA.read_capture(2)
mmcm2out  = scope.LA.read_capture(3)
glitchgo = scope.LA.read_capture(4)
glitchenable = scope.LA.read_capture(6)
glitchsource = scope.LA.read_capture(7)

In [ ]:
from bokeh.plotting import figure, show
from bokeh.resources import INLINE
from bokeh.io import output_notebook
from bokeh.models import Span, Legend, LegendItem
import numpy as np
output_notebook(INLINE)

o = figure(plot_width=1800)

xrange = range(len(source))
O1 = o.line(xrange, source + 6, line_color='black')
O2 = o.line(xrange, mmcm1out + 4, line_color='blue')
O3 = o.line(xrange, mmcm2out + 2, line_color='red')
O4 = o.line(xrange, glitchout + 0, line_color='purple', line_width=2)
O5 = o.line(xrange, glitchenable - 2, line_color='black', line_width=2)
#O6 = o.line(xrange, glitchgo - 4, line_color='green')
#O7 = o.line(xrange, glitchsource - 6, line_color='pink', line_width=2)

legend = Legend(items=[
    LegendItem(label='source clock', renderers=[O1]),
    LegendItem(label='glitch MMCM1 output', renderers=[O2]),
    LegendItem(label='glitch MMCM2 output', renderers=[O3]),
    LegendItem(label='glitch clock output', renderers=[O4]),
    LegendItem(label='glitch enable', renderers=[O5]),
    #LegendItem(label='glitch go', renderers=[O6]),
    #LegendItem(label='glitch trigger source', renderers=[O7]),
])
o.add_layout(legend)

In [ ]:
# add glitch markers:
def find_transitions(data, pattern):
    return [i for i in range(0,len(data)) if list(data[i:i+len(pattern)])==pattern]

transitions = [find_transitions(glitchout, [0,1])[0]+1, find_transitions(glitchout, [1,0])[0]]

for b in transitions:
    o.renderers.extend([Span(location=b, dimension='height', line_color='black', line_width=1, line_dash='dashed')])

In [ ]:
show(o)

### Interactive glitch visualization:

Now we step through many width/offset combinations so that we can interactively plot them.
We carry out STEPS * STEPS captures. STEPS can be whatever you want, but there it doesn't make sense to make STEPS greater that `scope.LA.oversampling_factor`.

In [ ]:
STEPS = 50
increment = scope.glitch.phase_shift_steps * 2 // STEPS
start = -scope.glitch.phase_shift_steps

import numpy as np
glitchouts = np.zeros((STEPS, STEPS, scope.LA.capture_depth))
sources    = np.zeros((STEPS, STEPS, scope.LA.capture_depth))
mmcm1outs  = np.zeros((STEPS, STEPS, scope.LA.capture_depth))
mmcm2outs  = np.zeros((STEPS, STEPS, scope.LA.capture_depth))
glitchenables = np.zeros((STEPS, STEPS, scope.LA.capture_depth))
glitchgo = np.zeros((STEPS, STEPS, scope.LA.capture_depth))

from tqdm import tnrange

scope.glitch.offset = start
scope.glitch.width = start

for o in tnrange(STEPS):
    scope.glitch.width = start
    for w in range(STEPS):
        scope.glitch.manual_trigger()
        glitchouts[o][w]   = scope.LA.read_capture(0)
        sources[o][w]      = scope.LA.read_capture(1)
        mmcm1outs[o][w]    = scope.LA.read_capture(2)
        mmcm2outs[o][w]    = scope.LA.read_capture(3)
        glitchgo[o][w]     = scope.LA.read_capture(4)
        glitchenables[o][w] = scope.LA.read_capture(6)

        scope.glitch.width += increment
    scope.glitch.width = start
    scope.glitch.offset += increment

In [ ]:
def update_plot(offset, width):
    S1.data_source.data['y'] = sources[offset][width] + 6
    S2.data_source.data['y'] = mmcm1outs[offset][width] + 4
    S3.data_source.data['y'] = mmcm2outs[offset][width] + 2 
    S4.data_source.data['y'] = glitchouts[offset][width] + 0
    S5.data_source.data['y'] = glitchenables[offset][width] - 2
    #S6.data_source.data['y'] = glitchgo[offset][width] - 4

    t1s = find_transitions(glitchouts[offset][width], [0,1])
    t2s = find_transitions(glitchouts[offset][width], [1,0])
    if len(t1s) == 1:
        T1.location = t1s[0]+1
    else:
        T1.location = 0

    if len(t2s) == 1:
        T2.location = t2s[0]
    else:
        T2.location = 0

    
    push_notebook()

In [ ]:
from ipywidgets import interact, Layout
from bokeh.io import push_notebook
from bokeh.models import Span, Legend, LegendItem

o = 0
w = 0

S = figure(plot_width=1800)

xrange = range(len(sources[o][w]))
S1 = S.line(xrange, sources[o][w] + 6, line_color='black')
S2 = S.line(xrange, mmcm1outs[o][w] + 4, line_color='blue')
S3 = S.line(xrange, mmcm2outs[o][w] + 2 , line_color='red')
S4 = S.line(xrange, glitchouts[o][w] + 0, line_color='purple', line_width=2)
S5 = S.line(xrange, glitchenables[o][w] - 2, line_color='green')
#S6 = S.line(xrange, glitchgo[o][w] - 4, line_color='pink', line_width=2)

t1s = find_transitions(glitchouts[o][w], [0,1])
t2s = find_transitions(glitchouts[o][w], [1,0])
if len(t1s) == 1:
    T1_location = t1s[0]+1
else:
    T1_location = 0

if len(t2s) == 1:
    T2_location = t2s[0]
else:
    T2_location = 0
        
T1 = Span(location=T1_location, dimension='height', line_color='black', line_dash='dashed', line_width=1)
T2 = Span(location=T2_location, dimension='height', line_color='black', line_dash='dashed', line_width=1)

legend = Legend(items=[
    LegendItem(label='source clock', renderers=[S1]),
    LegendItem(label='glitch MMCM1 output', renderers=[S2]),
    LegendItem(label='glitch MMCM2 output', renderers=[S3]),
    LegendItem(label='glitch clock output', renderers=[S4]),
    LegendItem(label='glitch enable', renderers=[S5]),
    #LegendItem(label='glitch go', renderers=[S6]),
])

S.add_layout(legend)
S.add_layout(T1)
S.add_layout(T2)

In [ ]:
show(S, notebook_handle=True)

In [ ]:
interact(update_plot, offset=(0, STEPS-1), width=(0, STEPS-1))

# Validation:

### For a particular static offset/width setting, runs lots of captures and ensure there are no missing glitches and no extra glitches:

In [ ]:
scope.glitch.repeat = 2
from tqdm import tnrange
import numpy as np
oversamp = scope.LA.oversampling_factor
lens = []
overlens = []
zerolens = 0
for i in tnrange(1000):
    scope.glitch.manual_trigger()
    glitchenable = scope.LA.read_capture(6)
    glitchlen = len(np.where(glitchenable > 0)[0])
    if not glitchlen:
        zerolens += 1
        continue
    lens.append(glitchlen)
    if abs(glitchlen/scope.glitch.repeat - oversamp) > oversamp / 4:
        overlens.append(glitchlen)

In [ ]:
assert zerolens == 0 and len(overlens) == 0

### Look for "double glitches" bug:

Here we just look at "go" length, for double glitches that can't otherwise be seen.
This takes a while.

In [ ]:
scope.clock.pll.update_fpga_vco(1200e6)

In [ ]:
reps = 2
width = -3000
oversamp = 30
stepsize = 1

scope.glitch.width = 0
scope.glitch.offset = 0

scope.clock.pll.update_fpga_vco(1200e6)

margin = 1
prev_offset = 0
scope.LA.oversampling_factor = oversamp
scope.glitch.width = width
scope.glitch.repeat = 1

overs = []
badoffsets = []

from tqdm import tnrange

for r in tnrange(reps):
    offsets = []
    glitches = []
    sources = []

    # sweep offset and check that glitch offset increases by expected amount each time:
    for i, o in enumerate(range(-scope.glitch.phase_shift_steps, scope.glitch.phase_shift_steps - stepsize, stepsize)):
        scope.glitch.offset = o
        scope.glitch.manual_trigger()
        glitchgo  = scope.LA.read_capture(4)
        golen = len(np.where(glitchgo > 0)[0])
        if golen and (abs(golen - oversamp) > oversamp/4):
            print("Go width exceeds margin: %d at offset=%d" % (golen, o))
            overs.append(golen)
            badoffsets.append(o)


In [ ]:
scope.clock.pll.update_fpga_vco(600e6)

When done, turn off MMCMs to cool down:

In [ ]:
scope.LA.enabled = False
scope.glitch.enabled = False